In [1]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder
from cosapp.drivers import NonLinearSolver, RunOnce, MonteCarlo, RunSingleCase
from cosapp.utils.distributions import Normal
import sys
sys.path.append('./model')
from Earth import Earth
import numpy as np

earth = Earth("earth")

l = 10 #Rocket's length on the plot
dt = 0.05 #Time-step


from cosapp.utils.distributions import Normal
from cosapp.drivers import MonteCarlo
from cosapp.recorders import DataFrameRecorder

In [6]:
def run_analysis(syst, draws=10, linear=True):
    syst.drivers.clear()  # Remove all drivers on the System

    runonce = syst.add_driver(RunOnce("runonce"))
    driver = syst.add_driver(RungeKutta(order=4, dt=dt))
    driver.add_child(NonLinearSolver('solver', factor=1.0))
    driver.time_interval = (0, 40)

    # Define a simulation scenario
    driver.set_scenario(
        init = {
        'Traj.r' : np.array([0., 0., l/2]),
        'Rocket.Kin.ar' : 'initrot',
        'Rocket.Kin.v' : np.array([0,0,0]),
        # 'Rocket.Kin.ar' : np.array([0., -np.pi/2 + 0.1, 0.0]),
        'Rocket.Kin.av' : np.zeros(3),
        'Para.DynPar.r1' : np.array([0., 0., l/2]), #(should be l because the parachute is at the tip of the rocket)
        'Para.DynPar.r2' : np.array([0., 0., l/2]),
        'Para.DynPar.v1' : np.array([0,0,0]),
        'Para.DynPar.v2' : np.array([0,0,0]),
        'Traj.ParaDepStatus': False,
        
    },
    stop='Para.DynPar.r1[2] < -1'
    )

    syst.run_drivers()
    print("MONTECARLO")
    # Montecarlo
    syst.drivers.clear()
    montecarlo = syst.add_driver(MonteCarlo('mc'))
    montecarlo.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r', 'initrot']))
    montecarlo.add_child(driver)
    montecarlo.draws = draws
    # montecarlo.linear = linear

    # parameters for uncertainties in the data
    pitch_attr = syst.inwards.get_details('pitch_init')
    yaw_attr = syst.inwards.get_details('yaw_init')
    
    # Set the distribution around the current value
    pitch_attr.distribution = Normal(best=.1, worst=-0.02)
    yaw_attr.distribution = Normal(best=.05, worst=-0.05)

    montecarlo.add_random_variable('pitch_init')
    montecarlo.add_random_variable('yaw_init')

    # Computation
    syst.run_drivers()

    return montecarlo.recorder.export_data()

In [7]:

results = run_analysis(earth, draws=2, linear=False)

MONTECARLO
___PARACHUTE DEPLOYMENT___


Point 1 X Coordinate at Deployment:  112.5856501438863 m


Point 1 Y Coordinate at Deployment:  1.1694872059807009 m


Point 1 Z Coordinate at Deployment:  807.359616558416 m


Point 2 X Coordinate at Deployment:  112.53656298349539 m


Point 2 Y Coordinate at Deployment:  1.1871594760651778 m


Point 2 Z Coordinate at Deployment:  806.862345865981 m


___PARACHUTE DEPLOYMENT___
Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Point 1 X Coordinate at Deployment:  114.60432894229778 m


Point 1 Y Coordinate at Deployment:  1.5212674658238834 m


Point 1 Z Coordinate at Deployment:  829.1198129691037 m


Point 2 X Coordinate at Deployment:  114.5549355694813 m


Point 2 Y Coordinate at Deployment:  1.5405200419550091 m


Point 2 Z Coordinate at Deployment:  828.6226312789399 m


___PARACHUTE DEPLOYMENT___
Parachute fully deployed !
Parachute fully deployed !
___PARACHUTE DEPLOYMENT___


Point 1 X Coordinate at Deployment:  114.50530936294619

In [8]:
results

,Section,Status,Error code,Reference,Rocket.Kin.ar,Traj.r,initrot
0,,,0,0,"[0.4209909584678674, -0.742397063651149, -0.38...","[-1322.6397532566054, 741.2026358120689, -2918...","[0.0, -1.4707963267948965, 0.04]"
1,,,0,1,"[2.2019615931222845, -1.2295404465711959, -2.1...","[635.2353915025869, 728.483206611399, -3235.40...","[0.0, -1.5033353088960768, 0.0790467785214164]"


In [16]:
traj = np.asarray(results['Traj.r'].tolist())
traj

array([[ -599.83199755,  2711.02561626, -1848.34171724],
       [ -601.61349086,  2525.91744493, -2100.02004344]])

In [14]:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

traj = np.asarray(results['Traj.r'].tolist())
print(traj[:, 0])

# Create the figure object
fig = make_subplots(rows=1, cols=1)
fig.layout.title = "Probability"
fig.layout.yaxis.title = 'Y Position'

fig.add_trace(
    go.Scatter(
        x=traj[:,0],
        y=traj[:,1],
        mode = 'markers'
    ),
    row = 1,
    col = 1,
)
fig.get_subplot(1, 1).xaxis.title = "X Position"

fig.show()

[-599.83199755 -601.61349086]
